In [1]:
import json as j
import pandas as pd
import re
import numpy
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import csv

data = pd.read_csv('iseardataset.csv')
test_data = data

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

data['cleaned'] = data['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
test_data['cleaned'] = data['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

data['cleaned'][1]

'everi time i imagin someon i love i could contact serious ill even death'

In [2]:
import numpy as np
wordsList = np.load('wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
SeqLength = 42
counter = 0
for p in wordsList:
    counter = counter+1
    if counter >= SeqLength:                
        break
        
wordVectors = np.load('wordVectors.npy')
wordVectors[21]

Loaded the word list!


array([ 0.35215 , -0.35603 ,  0.25708 , -0.10611 , -0.20718 ,  0.63596 ,
       -1.0129  , -0.45964 , -0.48749 , -0.080555,  0.43769 ,  0.46046 ,
       -0.80943 , -0.23336 ,  0.46623 , -0.10866 , -0.1221  , -0.63544 ,
       -0.73486 , -0.24848 ,  0.4317  ,  0.092264,  0.52033 , -0.46784 ,
        0.016798, -1.5124  , -0.19986 , -0.43351 , -0.59247 ,  0.18088 ,
        3.5194  , -0.7024  ,  0.23613 , -0.68514 , -0.37009 , -0.080451,
        0.10635 , -0.085495, -0.18451 ,  0.29771 ,  0.18123 ,  0.53627 ,
       -0.1001  , -0.55165 ,  0.098833, -0.12942 , -0.82628 , -0.4329  ,
       -0.10301 , -0.56079 ], dtype=float32)

In [3]:
labeltext = data['label']

labeltext1= data['label']


testdata = data['cleaned']

In [4]:
uniqueLabel=[]
for w in labeltext:
    if w not in uniqueLabel:
        uniqueLabel.append(w)
        
uniqueLabel

['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']

In [5]:
counter=[]
for word in uniqueLabel:
    count=0
    for word1 in labeltext:
        if word==word1:
            count=count+1
    counter.append(count)
    
sum(counter)

7516

In [6]:
list1 = data['cleaned']


len(list1)


list1[0]

'on day i feel close partner friend when i feel peac also experi close contact peopl i regard great'

In [7]:
numWords=[]
x = []
maxLengthWords = 0
for pf in list1:
    counter = len(pf.split())
    x.append(counter)
    numWords.append(counter)
    if(maxLengthWords < counter):
        maxLengthWords = counter

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))
print(maxLengthWords)

The total number of files is 7516
The total number of words in the files is 91532
The average number of words in the files is 12.178286322511974
95


In [8]:
maxSeqLength = 100

In [9]:
import tensorflow as tf
sess = tf.Session()

In [10]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [11]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0
for pf in list1:
    indexCounter = 0
    cleanedLine = cleanSentences(pf)
    split = cleanedLine.split()
    for word in split:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1 
    
np.save('idsMatrixRecall', ids)

In [12]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0
for pf in testdata:
    indexCounter = 0
    cleanedLine = cleanSentences(pf)
    split = cleanedLine.split()
    for word in split:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1 
    
np.save('idsMatrixRecallTest', ids)

In [13]:
ids1 = np.load('idsMatrixRecall.npy')


ids1[41]


ids = np.load('idsMatrixRecallTest.npy')
ids[0]

ids[41]

array([    61,     41,    998,   6710, 399999,     41,  90634,    530,
           41,   2716, 399999,  86821,   1927,   1671,     79, 399999,
        11056,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0])

In [14]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,7515)
        labeltxt = labeltext[num]
        if labeltxt=="joy":
            labels.append([1,0,0,0,0,0,0])
        if labeltxt=="fear":
            labels.append([0,1,0,0,0,0,0])
        if labeltxt=="anger":
            labels.append([0,0,1,0,0,0,0])
        if labeltxt=="sadness":
            labels.append([0,0,0,1,0,0,0])
        if labeltxt=="disgust":
            labels.append([0,0,0,0,1,0,0])
        if labeltxt=="shame":
            labels.append([0,0,0,0,0,1,0])
        if labeltxt=="guilt":
            labels.append([0,0,0,0,0,0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,7515)
        labeltxt = labeltext1[num]
        if labeltxt=="joy":
            labels.append([1,0,0,0,0,0,0])
        if labeltxt=="fear":
            labels.append([0,1,0,0,0,0,0])
        if labeltxt=="anger":
            labels.append([0,0,1,0,0,0,0])
        if labeltxt=="sadness":
            labels.append([0,0,0,1,0,0,0])
        if labeltxt=="disgust":
            labels.append([0,0,0,0,1,0,0])
        if labeltxt=="shame":
            labels.append([0,0,0,0,0,1,0])
        if labeltxt=="guilt":
            labels.append([0,0,0,0,0,0,1])
        arr[i] = ids1[num-1:num]
    return arr, labels

In [15]:
batchSize = 64
lstmUnits = 10
numClasses = 7
iterations = 100000
numDimensions = 300

In [16]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [17]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
data.shape

TensorShape([Dimension(64), Dimension(100), Dimension(50)])

In [18]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)
print(prediction)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
print(correctPred)
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
print(accuracy)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

Tensor("add:0", shape=(64, 7), dtype=float32)
Tensor("Equal:0", shape=(64,), dtype=bool)
Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [19]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [20]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
   #Next Batch of reviews
   nextBatch, nextBatchLabels = getTrainBatch()
   sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
   #Write summary to Tensorboard
   if (i % 50 == 0):
       summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
       writer.add_summary(summary, i)

   #Save the network every 10,000 training iterations
   if (i % 10000 == 0 and i != 0):
       save_path = saver.save(sess, "modelsRecall/pretrained_lstm.ckpt", global_step=i)
       print("saved to %s" % save_path)
writer.close()

saved to modelsRecall/pretrained_lstm.ckpt-10000
saved to modelsRecall/pretrained_lstm.ckpt-20000
saved to modelsRecall/pretrained_lstm.ckpt-30000
saved to modelsRecall/pretrained_lstm.ckpt-40000
saved to modelsRecall/pretrained_lstm.ckpt-50000
saved to modelsRecall/pretrained_lstm.ckpt-60000
saved to modelsRecall/pretrained_lstm.ckpt-70000
saved to modelsRecall/pretrained_lstm.ckpt-80000
saved to modelsRecall/pretrained_lstm.ckpt-90000


In [21]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('modelsRecall'))

INFO:tensorflow:Restoring parameters from modelsRecall\pretrained_lstm.ckpt-90000


In [23]:
iterations = 50
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch();
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 31.25
Accuracy for this batch: 29.6875
Accuracy for this batch: 43.75
Accuracy for this batch: 39.0625
Accuracy for this batch: 28.125
Accuracy for this batch: 31.25
Accuracy for this batch: 35.9375
Accuracy for this batch: 29.6875
Accuracy for this batch: 29.6875
Accuracy for this batch: 35.9375
Accuracy for this batch: 31.25
Accuracy for this batch: 28.125
Accuracy for this batch: 28.125
Accuracy for this batch: 40.625
Accuracy for this batch: 34.375
Accuracy for this batch: 31.25
Accuracy for this batch: 35.9375
Accuracy for this batch: 34.375
Accuracy for this batch: 35.9375
Accuracy for this batch: 42.1875
Accuracy for this batch: 42.1875
Accuracy for this batch: 43.75
Accuracy for this batch: 31.25
Accuracy for this batch: 34.375
Accuracy for this batch: 31.25
Accuracy for this batch: 42.1875
Accuracy for this batch: 32.8125
Accuracy for this batch: 23.4375
Accuracy for this batch: 42.1875
Accuracy for this batch: 43.75
Accuracy for this batch: 26.5625
Ac

In [24]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

In [25]:
inputText = "When I was not accepted as a student in finance and accounting."
inputMatrix = getSentenceMatrix(inputText)

In [26]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
max1 = 0
count = 0
for i in range(0,6):
    if predictedSentiment[max1] < predictedSentiment[i]:
        max1 = i
        count = predictedSentiment[i]

print(max1,count)
print(uniqueLabel[max1])
predictedSentiment

0 0
joy


array([ 3.4158468 ,  0.03969044, -0.48083097, -0.614553  ,  0.7445061 ,
       -1.1502123 ,  0.71592015], dtype=float32)

In [27]:
inputText = "When one lets friends down"
inputMatrix = getSentenceMatrix(inputText)

In [28]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
max1 = 0
count = 0
for i in range(0,6):
    if predictedSentiment[max1] < predictedSentiment[i]:
        max1 = i
        count = predictedSentiment[i]

print(max1,count)
print(uniqueLabel[max1])
predictedSentiment

5 0.6279112
shame


array([-0.31104717,  0.4194594 , -0.29769546,  0.5929631 , -0.26720387,
        0.6279112 ,  0.3004165 ], dtype=float32)

In [29]:
inputText = "When the whole family gets together for a one week holiday."
inputMatrix = getSentenceMatrix(inputText)

In [30]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
max1 = 0
count = 0
for i in range(0,6):
    if predictedSentiment[max1] < predictedSentiment[i]:
        max1 = i
        count = predictedSentiment[i]

print(max1,count)
print(uniqueLabel[max1])
predictedSentiment

0 0
joy


array([ 1.3058782 , -0.34141076,  0.472448  , -0.6915202 ,  0.8651525 ,
       -0.4371509 , -0.6871174 ], dtype=float32)

In [31]:
inputText = "On days when I feel close to my partner and other friends. When I feel at peace with myself and also experience a close contact with people whom I regard greatly."
inputMatrix = getSentenceMatrix(inputText)

In [40]:
inputArray = pd.read_csv('iseardataset.csv')
inputTextArray = inputArray["text"]
predictedArray=[]
inputLength = len(inputTextArray)
i=0
for text in inputTextArray:
    inputText = text
    inputMatrix = getSentenceMatrix(inputText)
    i=i+1
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    max1 = 0
    count = 0
    for j in range(0,6):
        if predictedSentiment[max1] < predictedSentiment[j]:
            max1 = j
            count = predictedSentiment[j]
    predictedArray.append(uniqueLabel[max1])
    if(i>=inputLength):
        break

IndexError: index 100 is out of bounds for axis 1 with size 100

In [39]:
predictedArray[0]
print(len(predictedArray))

2342


In [37]:
# Necessary imports: 
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

In [38]:
y_true = predictedArray
y_pred = inputArray['label']
print(precision_recall_fscore_support(y_true, y_pred, average='macro'))

ValueError: Found input variables with inconsistent numbers of samples: [2342, 7516]